<a href="https://colab.research.google.com/github/larasauser/master/blob/main/nspi_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NSPI 2

In [3]:
!pip install rasterio
!pip install numpy
!pip install opencv-python
!pip install tqdm

In [5]:
import numpy as np
import rasterio
from rasterio.plot import show
import cv2
import tqdm
from math import sqrt

In [7]:
def compute_rmsd(p1, p2):
    return np.sqrt(np.mean(p1-p2)**2)

In [10]:
def select_similar_pixels(input_patch, target_patch, gap_mask, stds, m, max_window):
  h,w,b = input_patch.shape
  x_c,y_c = h//2,w//2
  target_pixel = input_patch[x_c,y_c]

  similar_pixels = []
  distance = []

  for i in range(h):
    for j in range(w):
      if gap_mask[i,j]:
        continue
        candidate = input_patch[i,j]
        rmsd = compute_rmsd(candidate, target_pixel)
        threshold = np.sum([2*stds[k]/m for k in range(b)])/b
        if rmsd <= threshold:
          similar_pixels.append(((i,j),candidate))
          distances.append(np.linalg.norm([i-x_c, j-y_c]))
  return similar_pixels, distances

In [11]:
def predict_pixel_value(similar_pixels, distances, input_patch, target_patch, stds, m, max_window, input_central, target_central):
  if len(similar_pixels)==0:
    return None #use histogram matching if no similar pixel available

  rmsds = [compute_rmsd(input_patch[i,j],input_central) for (i,j),_ in similar_pixels]
  cds = 1/(np.array(cds)+1e-6)
  weights /=weights.sum()

  #Prediction L1 (eq7)
  L1 = np.zeros_like(input_central)
  for w, ((i.j),_) in zip(weights, similar_pixels):
    L1 += w*target_patch[i,j]

  #Prediction L2 (eq8)
  delta = np.zeros_like(input_central)
  for w, ((i,j),_) in zip(weights, similar_pixels):
    delta += w * (target_patch[i,j]-input_patch[i,j])
  L2 = input_central + delat

  #Final ponderation T1,T2 (eq11)
  R1 = np.mean([compute_rmsd(input_patch[i,j], input_central)for (i,j),_ in similar_pixels])
  R2 = np.mean([compute_rmsd(target_patch[i,j],input_patch[i,j])for (i,j),_ in similar_pixels])

  T1 = 1/R1 if R1 != 0 else 1e6
  T2 = 1/R2 if R2 != 0 else 1e6
  T1, T2 = T1/T_sum, T2/T_sum

  return T1*L1 + T2*L2

In [13]:
def filled_gaps_nspi(target, input_shape, gap_mask, m=20, max_window =150):
  filled = target.copy()
  h,w,b = target.shape
  pad = max_window//2
  padded_target = np.pad(target, ((pad,pad),(pad,pad),(0,0)),mode='reflect')
  padded_input = np.pad(input_image,((pad,pad),(pad,pad)(0,0)),mode='reflect')
  padded_mask = np.pad(gap.mask,((pad,pad),(pad,pad)),mode='constant',constant_values=1)

  stds = np.std(input_image[~gap_mask],axis=0)

  for y in tdqm(range(pad,pad+h)):
    for x in range(pad,pad+w):
      if not gap_mask[y-pad,x-pad]:
        continue

      for window in range(50, max_window +2, 2):
        r = window//2
        input_patch = padded_input[y-r:y+r+1, x-r:x+r+1]
        target_patch = padded_target[y-r:y+r+1, x-r:x+r+1]
        mask_patch = padded_mask[y-r:y+r+1, x-r:x+r+1]

        similar_pixels, distances = select_similar_pixels(
            input_patch, target_patch, mask_patch, stds, m, max_window
        )

        if len(similar_pixels) >= m or window == max_window:
          val = predict_pixel_value(similar_pixels, distances, input_patch, target_patch,stds,m,max_window,
                                    input_patch[r,r], target_patch[r,r])
          if val is not None:
            filled[y-pad,x-pad] = val
          break

  return filled


<>:6: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
<>:6: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
/tmp/ipython-input-13-1622071220.py:6: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
  padded_input = np.pad(input_image,((pad,pad),(pad,pad)(0,0)),mode='reflect')


Application